In [18]:
%load_ext autoreload
%autoreload 2

In [1]:
from pathlib import Path
from datetime import date 

folder =  Path("base")

In [2]:
import sys
sys.path.append(folder.absolute().as_posix())

In [3]:
from preprocess_utils import load_jsonb, save_jsonb 
from urllib.parse import urlparse
for art in load_jsonb("final.jsonb"):
    url = art["url"]
    parsed = urlparse(url).netloc.split(".")
    server = parsed[-1]
    if server == "cz":
        server = parsed[-2]

    save_jsonb([art], folder / f"{server}.jsonb", "a", show_progress=False)



    



KeyboardInterrupt: 

In [21]:
# Add from cz:403 to aktualne becauose of the same domain

# for art in load_jsonb(folder / "cz:443.jsonb"):
#     save_jsonb([art], folder / "aktualne.jsonb", "a", show_progress=False)

# Remove cz:443
import os
os.remove(folder / "cz:443.jsonb")
os.remove(folder / "ihned.jsonb")


In [7]:
from preprocess_utils import articles_num
print(list(articles_num(folder)))

[('ihned.jsonb', 31643), ('aktualne.jsonb', 165799), ('cz:443.jsonb', 1), ('denik.jsonb', 1636639), ('irozhlas.jsonb', 201681), ('novinky.jsonb', 549661), ('idnes.jsonb', 530855), ('seznamzpravy.jsonb', 77442)]


In [4]:
translate_cat = [
    (["zprávy z domova", "z domova", "domov"], "domácí"),
    (["zprávy ze světa", "svět", "zahraničí", "ze světa"], "zahraniční"),
    (["ona", "ženy", "pro ženy"], "žena"),
    (
        ["tipy deníku", "rádce", "rady učitele", "eko rady a tipy"],
        "tipy",
    ),
    (
        [
            "nepoužívat - věda",
            "věda a školy",
            "věda a technologie",
            "věda a technika",
            "věda a vesmír",
            "věda a příroda",
            "věda & vesmír"
        ],
        "věda",
    ),
    (
        ["technet", "tech & trendy", "tech", "technika"],
        "technologie",
    ),
    (
        ["osobní finance"],
        "finance",
    ),
    (
        [
            "nová auta",
            "automoto",
            "motorismus",
            "auta",
            "moje auto",
            "auta a motorky",
            "motocykly",
            "autohistorie",
        ],
        "auto",
    ),
    (
        ["televize", "film", "tv"],
        "film a tv",
    ),
    (["lidé"], "člověk"),
    (
        ["na kole po česku", "cyklorady", "cyklotrasy"],
        "kolo",
    ),
    (
        ["cestovánía dovolená", "po česku", "cestujeme"],
        "cestování",
    ),
    (
        ["rady a zajímavosti"],
        "zajímavosti",
    ),
    (["životní styl a společnost", "styl", "životnístyl"], "životní styl"),
    (["zdraví", "zdraví a fitness", "fit"], "zdraví"),
    (["nepouzivat - kultura"], "kultura"),
    (["jídelníček a recepty"], "recepty"),
    (["nepouzivat - kultura", "tipy na kulturu", "tipyna kulturu"], "kultura"),
    (["internet a pc"], "internet"),
    (["česká ekonomika", "světová ekonomika"], "ekonomika"),
    (["práce a podnikání", "podniky"], "podnikání"),
    (["hokej", "ms v hokeji", "ms hokej", "vancouver - hokej", "nhl", "extraliga"], "hokej"),
    (["kraje", "regiony"], "regiony"),
    (["ms ve fotbale", "česká liga", "liga mistrů", "evropská liga"], "fotbal"),
    (
        [
            "moje praha",
            "praha a střední čechy",
        ],
        "praha",
    ),
    (["klasická hudba"], "hudba"),
    (["basket"], "basketbal"),
    (["mobily", "telefony"], "mobil"),
    (["názory"], "komentáře"),
    (["loh 2020", "loh 2016 rio de janeiro", "tokio 2021", "tokio 2020", "loh 2012 londýn"], "olympijské hry"),
]


translate_cat = dict(
    [(x, y) for items, res in translate_cat for x, y in zip(items, [res] * len(items))]
)

with open("categories.txt") as f:
    cats = f.read().splitlines()

In [41]:
len(cats)

50

In [13]:
!pip install nltk

You should consider upgrading via the '/lnet/aic/personal/kydliceh/non_runable/NLP_venv/bin/python3 -m pip install --upgrade pip' command.


In [5]:

from typing import Callable, List
from filtering import create_filter_by_stats, create_filter_by_cz_lang, create_config, create_tokenized_filter
from preprocess_utils import load_jsonb, save_jsonb
from postprocessing_utils import postprocess_authors, filter_author, add_server, translate, filter_by_list, postprocess_brief, postprocess_category, postprocess_headline, postprocess_date, as_Article,JSONArticleEncoder, filter_by_list, add_cum_gender
filtered_folder = Path("article_json_filtered")

In [4]:
# Custom config for each server
from filtering import between
configs = {
    "idnes": {},
    "denik": {},
    "aktualne": {},
    "irozhlas": {},
    "seznamzpravy": {},
    "novinky": {},
}

default = {
    "article_length": between(300, None),
    "avg_word_length": between(4.0, None),
    "num_words_ratio": between(0.11, 0.22),
    "headline_length": between(20, None),
    "brief_length": between(40, None),
    "non_alpha_ratio": between(None, 0.045),
    "date": between(date(2000, 1, 1), date(2022, 8, 31)),
}

In [10]:

from functools import reduce
servers = [f for f in folder.iterdir()]

ratio = 0.75
for f in servers:
    filters = [
        create_filter_by_cz_lang(),
        create_filter_by_stats(create_config(configs[f.stem], default)),
    ]

    augmentations = [add_server(f.stem), filter_author,
    translate("category", translate_cat, lower=True),
    filter_by_list("category", cats, lower=True),
    postprocess_category,postprocess_authors, postprocess_brief, postprocess_headline, postprocess_date, add_cum_gender]


    articles = load_jsonb(f)
    articles = filter(lambda x: all(f(x) for f in filters), articles)


    articles = (reduce(lambda x, f: f(x), augmentations, article) for article in articles)
    


    as_articles = (as_Article(article) for article in articles)
    save_jsonb(as_articles, Path("test") / f"{f.stem}.jsonb", encoder=JSONArticleEncoder, show_progress=False)

         



    

100%|██████████| 165798/165798 [06:36<00:00, 418.18it/s]


In [10]:
# Create splits 70/20/10 by date
from datetime import date
import random
output_folder = Path("../final_dataset")
servers = [f for f in filtered_folder.iterdir()]
#servers = [fixed_folder / "seznamzpravy.jsonb"]

all_articles = (article for f in servers for article in load_jsonb(f))
none_articles = (article for article in all_articles if article["date"] is None)
articles = (article for article in all_articles if article["date"] is not None)
sorted_articles = sorted(articles, key=lambda x: date.fromisoformat(x["date"]))
# Randomly insert none articles
for none_art in none_articles:
    index = random.randint(0, len(sorted_articles))
    sorted_articles.insert(index, none_art)

train = sorted_articles[:int(len(sorted_articles) * 0.8)]
dev = sorted_articles[int(len(sorted_articles) * 0.8):int(len(sorted_articles) * 0.9)]
test = sorted_articles[int(len(sorted_articles) * 0.9):]

save_jsonb(train, output_folder / "train.jsonb", show_progress=True)
save_jsonb(dev, output_folder / "dev.jsonb", show_progress=True)
save_jsonb(test, output_folder / "test.jsonb", show_progress=True)



100%|██████████| 222746/222746 [00:26<00:00, 8385.78it/s]


In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from pathlib import Path
from preprocess_utils import load_jsonb, save_jsonb

In [3]:
for file in Path("../final_dataset/").iterdir():
    jss = []
    for js in load_jsonb(file):
        author_gen = js["author_genders"]
        gen_t = None
        if author_gen is not None:
            author_gen = [int(x) - 1 for x in author_gen]
            if all([a == 0 for a in author_gen]):
                gen_t = 0
            elif all([a == 1 for a in author_gen]):
                gen_t = 1
            else:
                gen_t = 2
            
        del(js["author_genders"])
        js["authors_gender"] = author_gen
        js["cum_gender"] = gen_t
        jss += [js]
    save_jsonb(jss, Path("../final") / f"{file.stem}.jsonb", show_progress=False)
        




        

100%|██████████| 222876/222876 [00:29<00:00, 7587.04it/s]
